# Assignment 2
---

#### A common task for a data scientist is to take a dataset and summarize the main features of the dataset.  Using that perspective we'll investigate the dataset of Benson et al., (2020).

> Benson, B., J. Magnuson, and S. Sharma. 2000, updated 2020. Global Lake and River Ice Phenology Database, Version 1. [Indicate subset used]. Boulder, Colorado USA. NSIDC: National Snow and Ice Data Center. https://doi.org/10.7265/N5W6

## Learning outcomes

#### 1. Be able to summarize important statistics of a data set
- Understand the distribution of observations over different lakes in the frozen lake dataset

#### 2. Fit a basic machine learning model using this dataset
- Conceptually understand the process of fitting a straight line

#### 3. Extract a climate signal from a real world dataset
- Calculate probabilities of lake freezing

---
## Instructions

1. Execute (shift-enter) each cell in this notebook in order.
2. _Skim read_ each cell:  read over each line to get a sense of what is going on.  If you don't understand something, just keep going and note the context from the cell output.  We'll discuss the code in more detail verbally during the lab.
3. Instructions that require you to do something (beyond just executing the cell) are written in **bold.**
4. Answer the questions in Assignment 2 (on Canvas)

---
## About Lake Suwa
From [Wikipedia](https://en.m.wikipedia.org/wiki/Lake_Suwa):
> Lake Suwa is the site of a natural phenomenon known as the "God's Crossing" (御神渡り, o-miwatari), large cracks that form in the winter across the surface of the frozen lake. A vertical temperature gradient results in ice pressure ridges forming in the surface ice, reaching heights of 30 centimetres (1 ft) or more.


![title](https://upload.wikimedia.org/wikipedia/commons/2/2b/Lake_Suwa_in_the_Shinano_province.jpg)
![Image](https://upload.wikimedia.org/wikipedia/commons/6/68/180205_Lake_Suwa_omiwatari_01.jpg)

# Part 1: Revisiting the frozen lake dataset

In [ ]:
# Download the dataset

import pandas as pd
import geopandas as gpd
import numpy as np

url_1="ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G01377/liag_freeze_thaw_table.csv"
df1=pd.read_csv(url_1)
df1 = df1[df1.longitude>-999]
lake_table = gpd.GeoDataFrame(df1, geometry=gpd.points_from_xy(df1.longitude, df1.latitude))

In [ ]:
# Plot the dataset as a map

import matplotlib.pyplot as plt
fig,ax = plt.subplots(figsize = (15,15))
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.plot(ax=ax)
lake_table.plot(ax = ax, markersize = 20, color = 'red',marker = '*',label = 'Delhi')
plt.show()

In [ ]:
# Show the dataset as a table. use head() to just show the first few lines

lake_table.head()

### How many observations are there for each lake? 
Pandas has a nice built-in function to answer this question, ```value_counts()```:

In [ ]:
number_of_observations = lake_table['lakename'].value_counts()
number_of_observations

In [ ]:
# Visualize the number of observations using a histogram
plt.subplots(figsize=(16,9))
plt.hist(number_of_observations,40)
plt.xlabel('Number of observations of an individual lake')
plt.ylabel('Number of lakes with X many observations')
plt.show()

In [ ]:
# Visualize the number of observations using a cumulative density plot
plt.subplots(figsize=(16,9))
plt.hist(number_of_observations,100,cumulative=True,density=True)
plt.xlabel('Number of lakes')
plt.ylabel('Fraction of lakes with at least X many observations')
plt.plot((-100,600),(0.5,0.5),'-r')
plt.plot((30,30),(0,1),'-r') # this is the answer to one of the quiz questions
plt.xlim([0,550])
plt.ylim([0,1.01])
plt.show()

**For the assignment, you'll need to do some calculations with the number_of_observations variable**. You can do that with the information avaialble up to this point in the notebook.  This is a good time to start working on the assignment.

In [ ]:
# Put all data from Lake Suwa in the variable suwa,
suwa = lake_table[lake_table.lakename == 'LAKE SUWA']
suwa.head()

# By looking at the first few entries, we'll recall that Lake Suwa 
# is unique because it has records going back to the year 1443

In [ ]:
def reformat_dates( lake_table ):
    '''
    Date variables are sort of awful in python. This function reformats dates.
    INPUT: a DataFrame of dates in the format of the frozen lake dataset
    OUTPUT: year, freeze_day; year is the year, freeze_day is the day that the lake froze,
        measured in the number of days since November 1 of a given year)
    '''
    import datetime

    year = []
    freeze_day= []

    for y,m,d in zip( lake_table.iceon_year,lake_table.iceon_month, lake_table.iceon_day ):
        if y > -999:
            days_from_nov1 = datetime.datetime(y,m,d) - datetime.datetime(y,11,1)
            freeze_day.append(days_from_nov1.total_seconds()/86400)
            year.append(y)

    year = np.array(year)
    freeze_day = np.array(freeze_day)
    return year, freeze_day

In [ ]:
# call the function
year, freeze_day = reformat_dates( suwa )

In [ ]:
# Plot the freezing days
plt.subplots(figsize=(16,9))
plt.plot(year,freeze_day,'o',label='Year with lake freeze',alpha=0.5)
plt.plot(year[freeze_day<0],-1*np.ones_like(year[freeze_day<0]),'or',label="Year without lake freeze",alpha=0.5)
plt.ylim([-2,70])
plt.xlim([1400,2100])
plt.ylabel('Days since Nobember 1')
plt.legend()
plt.show()

# Fit a simple machine learning model

The goal of "supervised" machine learning is to find a relationship between _features_ in the data and _labels_.

For our problem, let's just assume that the features are the _year_ and the _labels_ are the freezing day.

The simplest possible machine learning model consists of a straight line fit between pairs of (x,y) data. The process of fitting a straight line is called _linear regression_. 

In [ ]:
# Carry out linear regression

from scipy.stats import linregress

features = year[freeze_day>0]
labels = freeze_day[freeze_day>0]

output = linregress( features , labels)
output

In [ ]:
# Plot the result
plt.subplots(figsize=(16,9))
plt.plot(year,freeze_day,'o')
plt.plot(year,year*output.slope + output.intercept)
plt.ylim([0,70])
plt.xlim([1400,2100])
plt.ylabel('Days since Nobember 1')
plt.show()

How is the freezing day changing? The _slope_ of the straight line answers this question.  It gives us the answer in with units of "days per year" of change.  The numerical value of the slope is given in the output of the linear regression, above.

# How many ice free years?

With any machine learning model, it's always important to consider which data went into the analysis and which data was excluded.  In the above linear regression, we ignored years that the lake didn't freeze (because this doesn't fit neatly within the linear model).  

To account for the frozen versus non-frozen years, let's calculate the _probability_ of the lake freezing.

In [ ]:
number_of_freezing_years = sum(freeze_day>0)
total_number_of_years = len(freeze_day)
probability = number_of_freezing_years / total_number_of_years
print(probability)

In the assignment, you'll be asked to calculate the probability of freezing over a specific range of years.  **You can select certain years out of the dataframe using the syntax ```freeze_day[ some_condition ]```**.